# Retrieve Dataset

This notebook is used to capture a dataset.
The dataset contains RGB and depth images of both cameras, zivid and realsense.
Besides the images, the dataset stores the intrisic camera matrizes and distortion coefficients of both RGB lenses.

In [ ]:
# import dependencies
import time
import numpy as np
from pathlib import Path
# import required modules
from models.camera_interfaces.real_sense import RealSense
from models.camera_interfaces.zivid import Zivid
from models.dataset.dataset_interface import save_dataset

In [ ]:
resource_path = Path("../../resources")

class Args:
    picture_amount = 3                  # how many frames to collect
    delay = 500                         # delay between exposures in ms
    zivid_configuration_path: resource_path / Path("calibrations/zivid.yml") # zivid settings
    dataset_path = resource_path / Path("images/uncalibrated/dataset_0.npz") # where the captures dataset should be saved
args = Args()

# Connect to cameras

In [ ]:
# make sure both instances are destroyed. Otherwise they are still connected to the cameras
%reset_selective -f real_sense
%reset_selective -f zivid
# instantiate and connect to cameras
real_sense = RealSense()
RealSense.connect()

zivid = Zivid()
zivid.connect()
zivid.configure_manual(args.zivid_configuration_path)

print("Connected to both cameras")

rs_cm, rs_dc = real_sense.get_camera_matrix_and_distortion()
zv_cm, zv_dc = zivid.get_camera_matrix_and_distortion()

## Collect Pictures

In [ ]:
pictures = []

for i in range(args.picture_amount):
    rl_depth_image, rl_color_image = real_sense.collect_frame()
    z_depth_image, z_color_image = zivid.collect_frame()
    pictures.append((rl_depth_image, rl_color_image, z_depth_image, z_color_image))
    time.sleep(args.delay / 1000.0)

rs_rgb, rs_depth, zv_rgb, zv_depth = zip(*pictures)

print("Pictures Retrieved")

save_dataset(
    dataset_path=args.dataset_path, 
    realsense_color=rs_rgb,
    realsense_depth=rs_depth,
    realsense_cm=rs_cm,
    realsense_dc=rs_dc,
    zivid_color=zv_rgb,
    zivid_depth=zv_depth,
    zivid_cm=zv_cm,
    zivid_dc=zv_dc
)